# 📊 Constructor de Dataset: Blockchain de Bitcoin (BTC) / Métricas On-chain

**Fuente**: [Dataset Builder Bitcoin BTC Network On-Chain](https://www.kaggle.com/code/aleexharris/dataset-builder-bitcoin-btc-network-on-chain)

In [1]:
# =============================================================================
# LIBRERIAS
# =============================================================================

import requests
import time
import random
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from itertools import permutations
from typing import Callable

In [2]:
def to_snake_case(text: str) -> str:
    replacements = {' ': '_', '-': '_', '(': '', ')': '', ">": '', ',': ''}
    snake_case_text = text.lower().translate(str.maketrans(replacements)).replace("___", '_').replace("__", '_')
    return snake_case_text

## 🔹 Scraping BlockChain.com

In [3]:
def get_months_since_genesis() -> list[int]:
    genesis = datetime(2009, 1, 1)
    current_year = datetime.now().year
    years = np.arange(2009, current_year + 1, step=1)  # hasta el año actual incluido
    months = np.arange(1, 13, step=1)
    years, months = np.meshgrid(years, months)
    perms = np.column_stack((years.ravel(), months.ravel()))
    order = np.lexsort((perms[:, 1], perms[:, 0]))
    perms = perms[order]
    ts = [
        int(datetime(r[0], r[1], 1).timestamp())
        for r in perms
        if datetime(r[0], r[1], 1).date() < datetime.now().date()
    ]
    return ts

def blockchain_dot_com_get_request(endpoint: str, month: int) -> tuple[dict, str, str] | None:
    base_url= "https://api.blockchain.info/charts/"
    params = {"timespan": "5weeks", "format": "json", "start": month}
    for retry in range(5):
        try:
            r = requests.get(base_url + endpoint, params=params)
            data = r.json()["values"]
            desc = r.json()["description"]
            name = to_snake_case(r.json()["name"])
            return data, desc, name
        except Exception as e:
            time.sleep(random.randint(1, retry + 1))
            if retry == 4:
                msg = f"""Blockchain.com API endpoint seems broken...\n
                baseurl: {base_url}\n
                endpoint: {endpoint}\n
                params: {params}\n
                month: {datetime.from_timestamp(month)}\n
                exception: {type(e)}\n
                exception_args: {e.args}"""
                print(msg)
    else:
        return None

def get_blockchain_dot_com_endpoint_data(endpoint: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    column_data = []
    for month in get_months_since_genesis():
        response = blockchain_dot_com_get_request(endpoint, month)
        if not response:
            continue
        data, desc, name = response
        column_data += data
    column_df = pd.DataFrame(column_data).drop_duplicates().rename(columns={'x': "datetime", 'y': name})
    column_df["datetime"] = pd.to_datetime(column_df["datetime"], unit='s')
    describe_df = pd.DataFrame({"description": desc}, index=[name])
    return column_df, describe_df
    
def get_all_blockchain_dot_com_data() -> tuple[pd.DataFrame, pd.DataFrame]:
    endpoints = ["mempool-size", "transactions-per-second", "market-cap", "avg-block-size", "market-price",
                 "trade-volume", "avg-confirmation-time", "hash-rate", "difficulty", "miners-revenue", "transaction-fees"]
    timeseries_df = pd.Series([], dtype=int)
    info_df = pd.Series([], dtype=int)
    for e in tqdm(endpoints, desc="Blockchain.com endpoints scraped"):
        column_df, describe_df = get_blockchain_dot_com_endpoint_data(e)
        if timeseries_df.empty:
            timeseries_df = column_df
        else:
            timeseries_df = pd.merge(timeseries_df, column_df, on='datetime', how='outer')
        if info_df.empty:
            info_df = describe_df
        else:
            info_df = pd.concat([info_df, describe_df])
    return timeseries_df, info_df

bdc_timeseries_df, bdc_info_df = get_all_blockchain_dot_com_data()

Blockchain.com endpoints scraped: 100%|████████████████████████████████████████████████| 11/11 [07:43<00:00, 42.18s/it]


### 🧹 Limpieza de datos de Blockchain.com

- Se realizó en una sola función, ya que se requería muy poca limpieza.
- Se utilizó la mediana al reducir los datos de media hora a datos diarios para limitar el impacto de datos que no siguen una distribución normal.
- Se utilizó la mediana en los datos de media hora al remuestrear a nuevas series temporales de media hora para ignorar los valores NaN.
- **Nota**: El primer bloque de Bitcoin se minó el 3 de enero de 2009 por Satoshi, pero el segundo no se minó hasta el 9 de enero, 6 días después, de ah- í el período de datos nulos para las métricas on-chain entre el 3 y el 9 de enero de 2009.

In [4]:
def clean_blockchain_dot_com_data(ts_df: pd.DataFrame, info_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    rename_map = {"market_capitalization": "market_cap_usd", "usd_exchange_trade_volume": "exchange_volume_usd"}
    ts_df = ts_df.rename(columns=rename_map)
    daily_data = (ts_df.sort_values("datetime").groupby(ts_df.datetime.dt.date).median(numeric_only=False)
                  .iloc[:-1].fillna(0).drop("datetime", axis=1))
    half_hourly_data = (ts_df[["datetime", "transaction_rate", "mempool_size", "market_cap_usd"]]
                        .sort_values("datetime").set_index("datetime").resample("30T").median())
    info_data = info_df.rename(index=rename_map)
    info_data.at["average_confirmation_time", "description"] = """The average time taken for a transaction to be combined 
    in a Bitcoin block with other transactions and added to the blockchain."""  # missing from the API inexplicably
    return daily_data, half_hourly_data, info_data

bdc_daily_data, bdc_half_hourly_data, bdc_info_data = clean_blockchain_dot_com_data(bdc_timeseries_df, bdc_info_df)

C:\Users\misan\AppData\Local\Temp\ipykernel_29840\3581816446.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .sort_values("datetime").set_index("datetime").resample("30T").median())


In [5]:
bdc_daily_data

,mempool_size,transaction_rate,market_cap_usd,average_block_size,market_price_usd,exchange_volume_usd,average_confirmation_time,hash_rate,difficulty,miners_revenue,total_transaction_fees
datetime,,,,,,,,,,,
2009-01-03,0.0,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,4.971027e-08,1.000000e+00,0.000000e+00,0.000000
2009-01-04,0.0,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
2009-01-05,0.0,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
2009-01-06,0.0,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
2009-01-07,0.0,0.000000,0.000000e+00,0.000000,0.00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2025-08-11,1083776.5,3.808333,2.389988e+12,1.660835,119295.42,2.820060e+08,14.031899,9.458363e+08,1.294352e+14,5.591598e+07,4.415526
2025-08-12,947337.0,3.350000,2.373091e+12,1.570783,118715.93,5.619145e+08,16.722433,8.814937e+08,1.294352e+14,5.384354e+07,4.246542
2025-08-13,973646.0,3.750000,2.396955e+12,1.558120,120169.30,4.142620e+08,13.262250,1.003745e+09,1.294352e+14,6.004586e+07,4.397483


In [6]:
bdc_half_hourly_data

,transaction_rate,mempool_size,market_cap_usd
datetime,,,
2009-01-03 00:00:00,NaN,NaN,NaN
2009-01-03 00:30:00,NaN,NaN,NaN
2009-01-03 01:00:00,NaN,NaN,NaN
2009-01-03 01:30:00,NaN,NaN,NaN
2009-01-03 02:00:00,NaN,NaN,NaN
...,...,...,...
2025-08-16 11:00:00,3.175000,466293.50,2.342075e+12
2025-08-16 11:30:00,2.925000,492773.75,2.341282e+12
2025-08-16 12:00:00,3.016667,240240.25,NaN


In [7]:
bdc_info_data

,description
mempool_size,The aggregate size of transactions waiting to ...
transaction_rate,The number of Bitcoin transactions added to th...
market_cap_usd,The total USD value of bitcoin supply in circu...
average_block_size,The average block size in MB.
market_price_usd,Average USD market price across major bitcoin ...
exchange_volume_usd,The total USD value of trading volume on major...
average_confirmation_time,The average time taken for a transaction to be...
hash_rate,The estimated number of tera hashes per second...
difficulty,A relative measure of how difficult it is to f...
miners_revenue,Total value of coinbase block rewards and tran...
